In [9]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os, sys
from pathlib import Path
from math import floor
from random import randrange, random
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

from ML.dataset import PMUAngleDataset
from ML.models import SensorRNN
from ML.transforms import *
from ML.metrics import *
from ML.trainer import SensorTrainer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%%capture
import torch.backends.cudnn as cudnn
torch.cuda.empty_cache()
cudnn.benchmark = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bs = 64

# 2 load hyperparameters
sensor_num = 5
scada_size = 2

In [11]:
# Transformations
awgn = AWGN(60)
x1rescale = RescalePMU([-180, 49.5, 0, -180, 49.5, 0], [180, 50.5, 240, 180, 50.5, 240], sensor_num)
cut_time = CutoutTime(.3, 5)

x2rescale = Rescale(0, 240)
yrescale = Rescale(-180., 180.)

x1tfms = transforms.Compose([awgn, x1rescale])

In [12]:
# Set paths
scadapath = "2_load_scadadata"
datapath = "2_load_data"

In [13]:
# Define tests
# No augmentation 
noaug_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type=None),
    PMUAngleDataset(scadapath, datapath, x1tfms=transforms.Compose([x1rescale]), x2tfms=x2rescale, ytfms=yrescale),
    device
)

# Cutout (time)
cut_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type=None),
    PMUAngleDataset(scadapath, datapath, x1tfms=transforms.Compose([x1rescale, cut_time]), x2tfms=x2rescale, ytfms=yrescale),
    device
)

# GRU
gru_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type=None),
    PMUAngleDataset(scadapath, datapath, x1tfms=x1tfms, x2tfms=x2rescale, ytfms=yrescale),
    device
)

# LSTM
lstm_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type=None, rnn_type="lstm"),
    PMUAngleDataset(scadapath, datapath, x1tfms=x1tfms, x2tfms=x2rescale, ytfms=yrescale),
    device
)

# CNN-GRU
cnn_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type="cnn"),
    PMUAngleDataset(scadapath, datapath, x1tfms=x1tfms, x2tfms=x2rescale, ytfms=yrescale),
    device
)

# CNN-GRU (ResNet)
resnet_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type="resnet"),
    PMUAngleDataset(scadapath, datapath, x1tfms=x1tfms, x2tfms=x2rescale, ytfms=yrescale),
    device
)

# CNN-GRU (ResNet) with Attention
attn_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type="resnet", attention=True),
    PMUAngleDataset(scadapath, datapath, x1tfms=x1tfms, x2tfms=x2rescale, ytfms=yrescale),
    device
)

In [14]:
# Run tests
epochs = 150

noaug_results = noaug_test(epochs)
cut_results = cut_test(epochs)
gru_results = gru_test(epochs)
lstm_results = lstm_test(epochs)
cnn_results = cnn_test(epochs)
resnet_results = resnet_test(epochs)
attn_results = attn_test(epochs)

KeyboardInterrupt: 

In [ ]:
# Test 20 load with CNN-GRU (ResNet) with Attention
sensor_num = 21
scada_size = 14
x1rescale = RescalePMU([-180, 49.5, 0, -180, 49.5, 0], [180, 50.5, 240, 180, 50.5, 240], sensor_num)

large_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type="resnet", attention=True, num_layers=4),
    PMUAngleDataset("./scadadata", "./data", x1tfms=x1tfms, x2tfms=x2rescale, ytfms=yrescale),
    device
)

In [ ]:
# Run test
epochs = 150
large_results = large_test(epochs)

In [ ]:
large_first_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type="resnet", attention=True, one=True),
    PMUAngleDataset("./scadadata", "./data", x1tfms=x1tfms, x2tfms=x2rescale, ytfms=yrescale, idx=0),
    device
)
large_last_test = SensorTrainer(
    SensorRNN(input_size=30, batch_size=bs, device=device, sensor_num=sensor_num, scada_size=scada_size, cnn_type="resnet", attention=True, one=True),
    PMUAngleDataset("./scadadata", "./data", x1tfms=x1tfms, x2tfms=x2rescale, ytfms=yrescale, idx=-1),
    device
)

In [ ]:
large_first_results = large_first_test(epochs)
large_last_results = large_last_test(epochs)

In [ ]:
# Get best epoch and loss
print(min(noaug_results[2][10:]), noaug_results[2][10:].index(min(noaug_results[2])))
print(min(cut_results[2][10:]), cut_results[2][10:].index(min(cut_results[2])))
print(min(gru_results[2][10:]), gru_results[2][10:].index(min(gru_results[2])))
print(min(lstm_results[2][10:]), lstm_results[2][10:].index(min(lstm_results[2][10:])))
print(min(cnn_results[2][10:]), cnn_results[2][10:].index(min(cnn_results[2])))
print(min(resnet_results[2][10:]), resnet_results[2][10:].index(min(resnet_results[2])))
print(min(attn_results[2][10:]), attn_results[2][10:].index(min(attn_results[2])))

In [ ]:
# Rescale loss as angles
print(noaug_results[1][55] * 180)
print(cut_results[1][94] * 180)
print(gru_results[1][44] * 180)
print(lstm_results[1][89] * 180)
print(cnn_results[1][109] * 180)
print(resnet_results[1][127] * 180)
print(attn_results[1][43] * 180)

In [ ]:
# Generate plots
idx = 0

plt.figure(figsize=(10,7))
plt.plot(gru_results[idx])
plt.plot(lstm_results[idx])
plt.plot(cnn_results[idx])
plt.plot(resnet_results[idx])
plt.plot(attn_results[idx])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.yscale('log')
plt.title("Training Losses (L2)")
plt.legend(["gru", "lstm", "cnn", "resnet", "attn"], loc="upper right")

In [ ]:
idx = 2

plt.figure(figsize=(10,7))
plt.plot(gru_results[idx])
plt.plot(lstm_results[idx])
plt.plot(cnn_results[idx])
plt.plot(resnet_results[idx])
plt.plot(attn_results[idx])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.yscale('log')
plt.title("Validation Losses (L2)")
plt.legend(["gru", "lstm", "cnn", "resnet", "attn"], loc="upper right")

In [ ]:
idx = 1

fig, ax = plt.subplots(figsize=(10,7))
ax.plot([x * 180 for x in gru_results[idx]])
ax.plot([x * 180 for x in lstm_results[idx]])
ax.plot([x * 180 for x in cnn_results[idx]])
ax.plot([x * 180 for x in resnet_results[idx]])
ax.plot([x * 180 for x in attn_results[idx]])
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.set_yscale('log')
ax.set_title("Validation Losses (Mean Angle)")
ax.legend(["gru", "lstm", "cnn", "resnet", "attn"], loc="upper right")
ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
ax.set_yticks([2, 3, 5, 10, 20, 30, 50, 100])

In [ ]:
plt.plot(noaug_results[1])
plt.plot(cut_results[1])
plt.plot(awgn_results[1])
plt.plot(gru_results[1])
plt.xlabel("Epochs")
plt.ylabel("Mean Angle Error")
plt.title("Predicting Angles Together vs Seperately (SCADA 1)")
plt.legend(["noaug", "cut", "awgn", "all"], bbox_to_anchor=(1,1), loc="upper left")
plt.axis([0, 50, 0 , .04])

In [ ]:
plt.plot(np.stack(breakdowns, axis=0)[:,0])
plt.plot(loads1)
plt.xlabel("Epochs")
plt.ylabel("Mean Angle Error")
plt.title("Predicting Angles Together vs Seperately (SCADA 1)")
plt.legend(["Together", "Seperately"], bbox_to_anchor=(1,1), loc="upper left")
plt.axis([0, 90, 0 , 30])

In [ ]:
plt.plot(np.stack(breakdowns, axis=0)[:,-1])
plt.plot(loads14)
plt.xlabel("Epochs")
plt.ylabel("Mean Angle Error")
plt.title("Predicting Angles Together vs Seperately (SCADA 14)")
plt.legend(["Together", "Seperately"], bbox_to_anchor=(1,1), loc="upper left")
plt.axis([0, 100, 70 , 140])

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(large_results[3], label='_Hidden', color="grey")
ax.plot(np.stack(large_results[3], axis=0).mean(1), label="Average Angle", color="red")
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.set_title("Validation Loss (Angle) Breakdown for a Large Simulation")
ax.legend(loc="upper right")
ax.set_yscale('log')
ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
ax.set_yticks([2, 3, 5, 10, 20, 30, 50, 100, 150])